In [157]:
import pandas as pd

In [158]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [159]:
df = pd.read_csv('/content/drive/My Drive/male_players (legacy).csv') #df with Columns: 110 entries

<ipython-input-159-e48e0c66f557>:1: DtypeWarning: Columns (108) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/drive/My Drive/male_players (legacy).csv') #df with Columns: 110 entries


In [160]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11969 entries, 0 to 11968
Columns: 110 entries, player_id to player_face_url
dtypes: float64(59), int64(4), object(47)
memory usage: 10.0+ MB


## **Data preprocessing**

### **Dropping Columns with too many Null Values**

In [161]:
def preprocess_data(df):
    # Dropping columns with more than 1/3 NA's
    for column in df.columns:
        na_count = df[column].isna().sum()
        if na_count > (len(df) / 3):
            df.drop(column, axis='columns', inplace=True)

    # Dropping the first 7 columns
    df = df.iloc[:, 7:]

    # Dropping specific features
    columns_to_drop = [
        'league_name', 'league_level', 'club_team_id', 'club_name',
        'club_position', 'club_jersey_number', 'club_joined_date',
        'club_contract_valid_until_year', 'nationality_id', 'nationality_name',
        'real_face', 'player_positions', 'dob', 'work_rate', 'body_type'
    ]
    df.drop(columns=columns_to_drop, axis='columns', inplace=True)

    # Dropping the last 28 columns
    df = df.iloc[:, :-28]

    # Handling 'preferred_foot'
    preferred_foot = df['preferred_foot']
    df.drop('preferred_foot', axis=1, inplace=True)

    # Dropping columns with less than 0.5 correlation with overall score
    corr_matrix = df.corr()
    sorted_correlations = corr_matrix['overall'].sort_values(ascending=False)
    columns_to_drop = sorted_correlations[sorted_correlations < 0.5].index.tolist()
    df.drop(columns=columns_to_drop, inplace=True)

    # Adding 'preferred_foot' back
    df['preferred_foot'] = preferred_foot

    return df

In [162]:
df = preprocess_data(df)

In [163]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11969 entries, 0 to 11968
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   overall                   11969 non-null  int64  
 1   potential                 11968 non-null  float64
 2   value_eur                 11699 non-null  float64
 3   wage_eur                  11765 non-null  float64
 4   international_reputation  11968 non-null  float64
 5   passing                   10795 non-null  float64
 6   movement_reactions        11968 non-null  float64
 7   preferred_foot            11968 non-null  object 
dtypes: float64(6), int64(1), object(1)
memory usage: 748.2+ KB


### **Imputing the data**

In [164]:
from sklearn.impute import SimpleImputer
import pandas as pd

def impute_missing_values(df):
    # Splitting numeric and non-numeric data
    numeric_cols = df.select_dtypes(include=['number']).columns
    nonnumeric_cols = df.select_dtypes(exclude=['number']).columns

    # Impute na's that are numeric
    imputer = SimpleImputer(strategy="median")
    imputed_numeric = pd.DataFrame(imputer.fit_transform(df[numeric_cols]), columns=numeric_cols)

    # Impute na's that are non-numeric
    imputer_str = SimpleImputer(strategy="most_frequent")
    imputed_nonnumeric = pd.DataFrame(imputer_str.fit_transform(df[nonnumeric_cols]), columns=nonnumeric_cols)

    # Concatenate the imputed numeric and nonnumeric
    imputed_df = pd.concat([imputed_numeric, imputed_nonnumeric], axis=1)

    # Return the imputed DataFrame
    return pd.DataFrame(imputed_df, columns=df.columns)

In [165]:
df = impute_missing_values(df)

### **Label encoding the data**

In [166]:
from numpy import array
import numpy as np
from sklearn.preprocessing import LabelEncoder

def encode_preferred_foot(df):
    # Create a LabelEncoder object
    label_encoder = LabelEncoder()

    # Extract 'preferred_foot' values and encode them
    values = np.array(df['preferred_foot'])
    integer_encoded = label_encoder.fit_transform(values)

    # Drop the original 'preferred_foot' column
    df.drop('preferred_foot', axis=1, inplace=True)

    # Add the integer encoded 'preferred_foot' column back to the DataFrame
    df['preferred_foot'] = integer_encoded

    with open('preferred_foot_encoder.pkl', 'wb') as f:
        pickle.dump(label_encoder, f)

    return df

In [167]:
df = encode_preferred_foot(df)

### **Standard Scaling**

In [168]:
from sklearn.preprocessing import StandardScaler

def prepare_features_and_target(df):
    # Selecting features and target
    X = df.drop(['overall'], axis=1)
    y = df['overall']

    # Standard Scaling
    sc = StandardScaler()
    X_scaled = sc.fit_transform(X)

    return X_scaled, y

In [169]:
X, y = prepare_features_and_target(df)

## **Training models**



In [170]:
from sklearn.model_selection import train_test_split

# Splitting data into training and testing sets
Xtrain, Xtest, Ytrain, Ytest = train_test_split(X, y, test_size=0.2, random_state=42)


In [171]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

def train_and_evaluate_models(Xtrain, Ytrain, Xtest, Ytest):
    models = {
        'RandomForestRegressor': RandomForestRegressor(n_estimators=100, random_state=42),
        'XGBRegressor': XGBRegressor(objective='reg:squarederror', random_state=42),
        'GradientBoostingRegressor': GradientBoostingRegressor(random_state=42)
    }

    results = {}

    for name, model in models.items():
        # Train the model
        model.fit(Xtrain, Ytrain)

        # Make predictions
        y_pred = model.predict(Xtest)

        # Calculate metrics
        mse = mean_squared_error(Ytest, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(Ytest, y_pred)
        r2 = r2_score(Ytest, y_pred)

        # Store results
        results[name] = {
            'model': model,
            'RMSE': rmse,
            'MAE': mae,
            'R2': r2
        }

        # Print results
        print(f"\nResults for {name}:")
        print(f"Root Mean Square Error: {rmse}")
        print(f"Mean Absolute Error: {mae}")
        print(f"R2 Score: {r2}")

    return results

In [172]:
results = train_and_evaluate_models(Xtrain, Ytrain, Xtest, Ytest)


Results for RandomForestRegressor:
Root Mean Square Error: 0.9785906156422566
Mean Absolute Error: 0.6143547360464654
R2 Score: 0.9670659223313272

Results for XGBRegressor:
Root Mean Square Error: 0.9946746277989036
Mean Absolute Error: 0.667246209051376
R2 Score: 0.9659744235100505

Results for GradientBoostingRegressor:
Root Mean Square Error: 1.0925285207580862
Mean Absolute Error: 0.7783317068402139
R2 Score: 0.9589503953953633


## **Grid Search with Cross Validation**

### **Gradient Boosting Regressor**

In [89]:
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pickle as pkl
import os

# Setting up cross-validation and model parameters
cv = KFold(n_splits=3)
gbr = GradientBoostingRegressor(random_state=42)
PARAMETERS_gb = {
    "max_depth": [2, 5, 6],
    "learning_rate": [0.3, 0.1, 0.03],
    "n_estimators": [100, 500, 1000]
}

# Performing grid search with cross-validation
model_gs = GridSearchCV(gbr, param_grid=PARAMETERS_gb, cv=cv, scoring="neg_mean_squared_error")
model_gs.fit(Xtrain, Ytrain)

# Saving the model
model_filename = 'GradientBoostingRegressor_GridSearch.pkl'
try:
    with open(model_filename, 'wb') as file:
        pkl.dump(model_gs, file)
    print(f"Model saved successfully as {model_filename}")
except Exception as e:
    print(f"Error saving the model: {e}")

# Making predictions
y_pred = model_gs.predict(Xtest)

# Printing regression metrics
print(model_gs.best_estimator_.__class__.__name__)
print("Best parameters found: ", model_gs.best_params_)
print("Mean Squared Error:", mean_squared_error(Ytest, y_pred))
print("Mean Absolute Error:", mean_absolute_error(Ytest, y_pred))
print("R^2 Score:", r2_score(Ytest, y_pred))

# Verifying if the file was created
if os.path.exists(model_filename):
    print(f"File {model_filename} exists.")
    print(f"File size: {os.path.getsize(model_filename)} bytes")
else:
    print(f"File {model_filename} does not exist.")

Model saved successfully as GradientBoostingRegressor_GridSearch.pkl
GradientBoostingRegressor
Best parameters found:  {'learning_rate': 0.03, 'max_depth': 5, 'n_estimators': 1000}
Mean Squared Error: 0.950367113815074
Mean Absolute Error: 0.6640856444619391
R^2 Score: 0.9673160293620156
File GradientBoostingRegressor_GridSearch.pkl exists.
File size: 3899323 bytes


### **XGB Regressor**

In [131]:
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from xgboost import XGBRegressor
import pickle as pkl
import os

# Setting up cross-validation and model parameters
cv = KFold(n_splits=3)
xgbr = XGBRegressor(random_state=42)
PARAMETERS_xgb = {
    "max_depth": [2, 5, 6],
    "learning_rate": [0.3, 0.1, 0.03],
    "min_child_weight": [1,5,15],
    "n_estimators": [100, 500, 1000]
}

# Performing grid search with cross-validation
model_gs = GridSearchCV(xgbr, param_grid=PARAMETERS_xgb, cv=cv, scoring="neg_mean_squared_error")
model_gs.fit(Xtrain, Ytrain)

# Saving the model
model_filename = 'XGBRegressor_GridSearch.pkl'
try:
    with open(model_filename, 'wb') as file:
        pkl.dump(model_gs, file)
    print(f"Model saved successfully as {model_filename}")
except Exception as e:
    print(f"Error saving the model: {e}")

# Making predictions
y_pred = model_gs.predict(Xtest)

# Printing regression metrics
print(model_gs.best_estimator_.__class__.__name__)
print("Best parameters found: ", model_gs.best_params_)
print("Mean Squared Error:", mean_squared_error(Ytest, y_pred))
print("Mean Absolute Error:", mean_absolute_error(Ytest, y_pred))
print("R^2 Score:", r2_score(Ytest, y_pred))

# Verifying if the file was created
if os.path.exists(model_filename):
    print(f"File {model_filename} exists.")
    print(f"File size: {os.path.getsize(model_filename)} bytes")
else:
    print(f"File {model_filename} does not exist.")

Model saved successfully as XGBRegressor_GridSearch.pkl
XGBRegressor
Best parameters found:  {'learning_rate': 0.03, 'max_depth': 6, 'min_child_weight': 1, 'n_estimators': 1000}
Mean Squared Error: 0.9371518515556908
Mean Absolute Error: 0.6505846630660513
R^2 Score: 0.9677705139894612
File XGBRegressor_GridSearch.pkl exists.
File size: 3869565 bytes


### **RandomForest Regressor**

In [91]:
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import pickle as pkl
import os

# Setting up cross-validation and model parameters
cv = KFold(n_splits=3)
rfr = RandomForestRegressor(random_state=42)
PARAMETERS_rf = {
    "max_depth": [2, 5, 6],
    "n_estimators": [100, 500, 1000],
    "max_features": ["auto", "sqrt", "log2"]
}

# Performing grid search with cross-validation
model_gs = GridSearchCV(rfr, param_grid=PARAMETERS_rf, cv=cv, scoring="neg_mean_squared_error")
model_gs.fit(Xtrain, Ytrain)

# Saving the model
model_filename = 'RandomForestRegressor_GridSearch.pkl'
try:
    with open(model_filename, 'wb') as file:
        pkl.dump(model_gs, file)
    print(f"Model saved successfully as {model_filename}")
except Exception as e:
    print(f"Error saving the model: {e}")

# Making predictions
y_pred = model_gs.predict(Xtest)

# Printing regression metrics
print(model_gs.best_estimator_.__class__.__name__)
print("Best parameters found: ", model_gs.best_params_)
print("Mean Squared Error:", mean_squared_error(Ytest, y_pred))
print("Mean Absolute Error:", mean_absolute_error(Ytest, y_pred))
print("R^2 Score:", r2_score(Ytest, y_pred))

# Verifying if the file was created
if os.path.exists(model_filename):
    print(f"File {model_filename} exists.")
    print(f"File size: {os.path.getsize(model_filename)} bytes")
else:
    print(f"File {model_filename} does not exist.")

/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/usr/local/lib/python3.10/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors

Model saved successfully as RandomForestRegressor_GridSearch.pkl
RandomForestRegressor
Best parameters found:  {'max_depth': 6, 'max_features': 'sqrt', 'n_estimators': 1000}
Mean Squared Error: 1.3769552495922106
Mean Absolute Error: 0.8591297291659842
R^2 Score: 0.9526452838137163
File RandomForestRegressor_GridSearch.pkl exists.
File size: 8274166 bytes


### **Training again on the best model**

In [132]:
from xgboost import XGBRegressor
import pickle as pkl

# Create the best XGBoost model with the optimal parameters
best_xgb_model = XGBRegressor(
    learning_rate=0.03,
    max_depth=6,
    n_estimators=1000,
    min_child_weight= 1,
    random_state=42  # for reproducibility
)

# Fit the model on the entire training data
best_xgb_model.fit(Xtrain, Ytrain)

# Save the model
best_model_filename = 'Best_XGBRegressor.pkl'
try:
    with open(best_model_filename, 'wb') as file:
        pkl.dump(best_xgb_model, file)
    print(f"Best model saved successfully as {best_model_filename}")
except Exception as e:
    print(f"Error saving the best model: {e}")

# Verify if the file was created
import os
if os.path.exists(best_model_filename):
    print(f"File {best_model_filename} exists.")
    print(f"File size: {os.path.getsize(best_model_filename)} bytes")
else:
    print(f"File {best_model_filename} does not exist.")

# Evaluate the best model on the test set
y_pred_best = best_xgb_model.predict(Xtest)

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
import numpy as np

# Calculate and print metrics
mse = mean_squared_error(Ytest, y_pred_best)
rmse = np.sqrt(mse)
mae = mean_absolute_error(Ytest, y_pred_best)
r2 = r2_score(Ytest, y_pred_best)

print("\nBest XGBoost Model Performance:")
print(f"Root Mean Square Error: {rmse}")
print(f"Mean Absolute Error: {mae}")
print(f"R^2 Score: {r2}")

Best model saved successfully as Best_XGBRegressor.pkl
File Best_XGBRegressor.pkl exists.
File size: 3857687 bytes

Best XGBoost Model Performance:
Root Mean Square Error: 0.9680660367741918
Mean Absolute Error: 0.6505846630660513
R^2 Score: 0.9677705139894612


## **Testing with players_22 data**

In [173]:
import pandas as pd

# Load the new data
players = pd.read_csv('/content/drive/My Drive/players_22.csv') #df with Columns: 110 entries)


<ipython-input-173-a50ce2352cba>:4: DtypeWarning: Columns (25,108) have mixed types. Specify dtype option on import or set low_memory=False.
  players = pd.read_csv('/content/drive/My Drive/players_22.csv') #df with Columns: 110 entries)


In [174]:
# Get the columns that are in df but not in new_data
columns_to_keep = [col for col in df.columns if col in players.columns]

# Keep only these columns in new_data
players = players[columns_to_keep]

In [175]:
players = impute_missing_values(players)

In [176]:
# Create a LabelEncoder object
label_encoder = LabelEncoder()

# Extract 'preferred_foot' values and encode them
values = np.array(players['preferred_foot'])
integer_encoded = label_encoder.fit_transform(values)

# Drop the original 'preferred_foot' column
players.drop('preferred_foot', axis=1, inplace=True)

# Add the integer encoded 'preferred_foot' column back to the DataFrame
players['preferred_foot'] = integer_encoded

In [178]:
# Prepare features (X) and target (y) for the new data
X_new = players.drop(['overall'], axis=1)
y_new = players['overall']

In [179]:
# Apply standard scaling
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_new_scaled = sc.fit_transform(X_new)

In [180]:
import pickle as pkl

# Load the models
with open('Best_XGBRegressor.pkl', 'rb') as file:
    best_model = pkl.load(file)

# Make predictions
xgb_predictions = best_model.predict(X_new_scaled)

In [181]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

mse = mean_squared_error(y_new, xgb_predictions)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_new, xgb_predictions)
r2 = r2_score(y_new, xgb_predictions)

print(f"\nResults for XGBoost:")
print(f"Root Mean Square Error: {rmse}")
print(f"Mean Absolute Error: {mae}")
print(f"R2 Score: {r2}")



Results for XGBoost:
Root Mean Square Error: 3.87950946110754
Mean Absolute Error: 2.8352580859399747
R2 Score: 0.6820417474070294
